In [1]:
%%time

import pathlib
from os.path import expanduser, join, abspath

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as fn
from pyspark.sql.types import *
from pyspark.sql.window import Window

warehouse_location = abspath('/home/jovyan/work/hive-db/spark-warehouse')
#warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
        .builder \
        .config("spark.sql.warehouse.dir", warehouse_location) \
        .enableHiveSupport() \
        .appName("local-test") \
        .getOrCreate()

spark

CPU times: user 407 ms, sys: 246 ms, total: 652 ms
Wall time: 10.7 s


In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import dask.dataframe as dd

- テストデータ用意

In [3]:
iris = sns.load_dataset('iris')
df_iris = spark.createDataFrame(iris)
df_iris.show()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         4.6|        3.4|         1.4|        0.3| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         4.4|        2.9|         1.4|        0.2| setosa|
|         4.9|        3.1|         1.5|        0.1| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         4.8|        3.4|         1.6|        0.2| setosa|
|         4.8|        3.0|         1.4|        0.1| setosa|
|         4.3|        3.0|         1.1| 

In [4]:
%%time
# DataFrame-APIから永続テーブルに保存
df_iris.write.saveAsTable("iris")

CPU times: user 3.71 ms, sys: 0 ns, total: 3.71 ms
Wall time: 8.29 s


- 下記リンクなどを参考にしているが、hive-site.xmlの設定はうまくいっていない
    - Notebookを実行しているディレクトリ上に`metastore_db/`と`derby.log`が設定されてしまう（←生成場所を制御したい）
    - https://software.fujitsu.com/jp/manual/manualfiles/m160007/j2ul2025/02z200/j2025-c-03-00.html

- データベース生成

In [10]:
spark.sql("""
create database if not exists sns
""")

DataFrame[]

- `/path/to/spark-datawarehouse/sns.db/`が生成される

In [15]:
spark.sql("""
create database if not exists ext
location '/home/jovyan/work/hive-db/spark-warehouse/ext.db'
""")

DataFrame[]

In [16]:
spark.sql("""
describe database ext
""").toPandas()

,database_description_item,database_description_value
0,Database Name,ext
1,Description,
2,Location,file:/home/jovyan/work/hive-db/spark-warehouse...


In [17]:
#spark.sql("drop database ext") 削除

In [19]:
spark.sql("""
create table if not exists sns.iris
as select * from iris
""")

spark.sql("""
select * from sns.iris
""").show()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         4.6|        3.4|         1.4|        0.3| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         4.4|        2.9|         1.4|        0.2| setosa|
|         4.9|        3.1|         1.5|        0.1| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         4.8|        3.4|         1.6|        0.2| setosa|
|         4.8|        3.0|         1.4|        0.1| setosa|
|         4.3|        3.0|         1.1| 

In [21]:
spark.sql("""
describe table sns.iris
""").show()

+------------+---------+-------+
|    col_name|data_type|comment|
+------------+---------+-------+
|sepal_length|   double|   null|
| sepal_width|   double|   null|
|petal_length|   double|   null|
| petal_width|   double|   null|
|     species|   string|   null|
+------------+---------+-------+



In [30]:
%%time
# DataFrame-APIから永続テーブルを保存
flights = sns.load_dataset('flights')
df_flights = spark.createDataFrame(flights)
df_flights.write.saveAsTable('sns.flights')

spark.sql("""
select * from sns.flights
""").show()

+----+---------+----------+
|year|    month|passengers|
+----+---------+----------+
|1955|  January|       242|
|1955| February|       233|
|1955|    March|       267|
|1955|    April|       269|
|1955|      May|       270|
|1955|     June|       315|
|1955|     July|       364|
|1955|   August|       347|
|1955|September|       312|
|1955|  October|       274|
|1955| November|       237|
|1955| December|       278|
|1956|  January|       284|
|1956| February|       277|
|1956|    March|       317|
|1956|    April|       313|
|1956|      May|       318|
|1956|     June|       374|
|1956|     July|       413|
|1956|   August|       405|
+----+---------+----------+
only showing top 20 rows

CPU times: user 39.7 ms, sys: 0 ns, total: 39.7 ms
Wall time: 652 ms


In [28]:
spark.sql("""
show tables
""").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  flights|      false|
| default|     iris|      false|
+--------+---------+-----------+



In [27]:
spark.sql("""
show tables from sns
""").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|     sns|  flights|      false|
|     sns|     iris|      false|
+--------+---------+-----------+



In [39]:
spark.sql("""
create table sns.flights2
-- location '/home/jovyan/work/hive-db/spark-warehouse/sns.db/flights2'
using parquet
partitioned by ( pt1,  pt2 )
as select *, year as pt1, month as pt2 from flights
""")

spark.sql("""
select * from sns.flights2
""")

spark.sql("""
describe table sns.flights2
""")

DataFrame[col_name: string, data_type: string, comment: string]

- `using parquet`などを使わないとうまく動かない